In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [4]:
# Load datasets
brent = pd.read_csv(r"C:\Users\Stevi\OneDrive\Documents\Projects\Crude-Oil\Data ingestion\Brent.csv")
wti = pd.read_csv(r"C:\Users\Stevi\OneDrive\Documents\Projects\Crude-Oil\Data ingestion\WTI.csv")

nifty50 = pd.read_csv(r"C:\Users\Stevi\OneDrive\Documents\Projects\Crude-Oil\Data ingestion\Nifty50.csv")
nifty100 = pd.read_csv(r"C:\Users\Stevi\OneDrive\Documents\Projects\Crude-Oil\Data ingestion\Nifty100.csv")
nifty200 = pd.read_csv(r"C:\Users\Stevi\OneDrive\Documents\Projects\Crude-Oil\Data ingestion\Nifty200.csv")
nifty500 = pd.read_csv(r"C:\Users\Stevi\OneDrive\Documents\Projects\Crude-Oil\Data ingestion\Nifty500.csv")

In [5]:
# Preview top rows and columns
datasets = {
    "Brent": brent,
    "WTI": wti,
    "Nifty50": nifty50,
    "Nifty100": nifty100,
    "Nifty200": nifty200,
    "Nifty500": nifty500
}

In [6]:
for name, df in datasets.items():
    print(f"\n📁 {name} — Shape: {df.shape}")
    print(df.head(3))


📁 Brent — Shape: (3142, 8)
         Date   BZ=F_Open   BZ=F_High    BZ=F_Low  BZ=F_Close  BZ=F_Adj Close  \
0  2013-01-02  111.940002  112.900002  111.760002  112.470001      112.470001   
1  2013-01-03  111.919998  112.389999  111.699997  112.139999      112.139999   
2  2013-01-04  110.720001  111.739998  110.720001  111.309998      111.309998   

   BZ=F_Volume Ticker  
0         1343  Brent  
1          752  Brent  
2         1279  Brent  

📁 WTI — Shape: (3152, 8)
         Date       Open       High        Low      Close  Adj Close  Volume  \
0  2013-01-02  91.779999  93.870003  91.559998  93.120003  93.120003  203865   
1  2013-01-03  92.910004  93.300003  92.489998  92.919998  92.919998  189812   
2  2013-01-04  92.860001  93.209999  91.519997  93.089996  93.089996  210747   

  Ticker  
0    WTI  
1    WTI  
2    WTI  

📁 Nifty50 — Shape: (3079, 8)
         Date   ^NSEI_Open   ^NSEI_High    ^NSEI_Low  ^NSEI_Close  \
0  2013-01-02  5982.600098  6006.049805  5982.000000  5993.25

In [9]:
# Clean and rename
brent = brent[["Date", "BZ=F_Close"]].rename(columns={"Close": "brent_price"})
nifty50 = nifty50[["Date", "^NSEI_Close"]].rename(columns={"Close": "nifty_price"})

In [11]:
# Merge on Date
df = pd.merge(brent, nifty50, on="Date")
df = df.sort_values("Date").dropna()

In [13]:
df

,Date,BZ=F_Close,^NSEI_Close
0,2013-01-02,112.470001,5993.250000
1,2013-01-03,112.139999,6009.500000
2,2013-01-04,111.309998,6016.149902
3,2013-01-07,111.400002,5988.399902
4,2013-01-08,111.940002,6001.700195
...,...,...,...
2977,2025-07-09,70.190002,25476.099609
2978,2025-07-10,68.639999,25355.250000
2979,2025-07-11,70.360001,25149.849609
2980,2025-07-14,69.209999,25082.300781


In [12]:
# Calculate returns and volatility
df['brent_ret'] = df['brent_price'].pct_change()
df['nifty_ret'] = df['nifty_price'].pct_change()
df['brent_vol_10d'] = df['brent_ret'].rolling(10).std()
df['nifty_vol_10d'] = df['nifty_ret'].rolling(10).std()
df = df.dropna().reset_index(drop=True)

KeyError: 'brent_price'